# Train classifer on date phrases to find plan start date

In [2]:
import csv
import pandas as pd
import os
import classify
import extract_dates
import random

## Create training data

In [3]:
docs = pd.read_csv(os.path.join(data_path, 'text_narrowed.csv'))

### Randomly sample five plans

Do not overwrite labelled document set unless you want to relabel. relabel_docs = False uses previously randomly sample plans.

In [4]:
relabel = False
if relabel:
    docs_to_label = docs[docs.p_innovation >.5].sample(n = 6)
    docs_to_label.index
    docs_to_label.head(5)

In [5]:
if not relabel:
    docs_to_label = docs[docs.title == "Ector County ISD"]
    docs_to_label = docs_to_label.append(docs[docs.title == "Dimmitt ISD"])
    docs_to_label = docs_to_label.append(docs[docs.title == "Sunnyvale ISD"])
    docs_to_label = docs_to_label.append(docs[docs.title == "Sunray ISD"])
    docs_to_label = docs_to_label.append(docs[docs.title == "Douglass ISD"])
    docs_to_label = docs_to_label.append(docs[docs.title == "Dodd City ISD"])
    docs_to_label = docs_to_label.append(docs[docs.title == "Snyder ISD"])
    docs_to_label = docs_to_label.append(docs[docs.title == "Mart ISD"])
    docs_to_label = docs_to_label.append(docs[docs.title == "Rivercrest ISD"])
    docs_to_label.head(5)

In [6]:
docs_to_label

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,title,level,type,link,text,p_innovation
568,270,270,270,Ector County ISD,First,pdf,http://www.ectorcountyisd.org/cms/lib011/tx010...,ECISD District of Innovation Plan 1 I. Introdu...,0.999667
579,3631,3631,288,Dimmitt ISD,html,html,http://www.dimmittisd.net/vnews/display.v/ART/...,Dimmitt ISD - District of Innovation Quick Lin...,0.986358
101,3331,3331,3331,Sunnyvale ISD,Second,pdf,http://www.sunnyvaleisd.com/site/handlers/file...,Sunnyvale ISD District of Innovation Plan Spri...,0.999955
100,98,98,98,Sunray ISD,First,pdf,https://1.cdn.edl.io/Rb3eNoIDEma2NVfd5LdmKpqLB...,SUNRAY ISD District of Innovation Plan 2 Distr...,0.999829
577,291,291,291,Douglass ISD,First,google,https://drive.google.com/file/d/1_OzEr7LnngRcQ...,Douglass ISD District of Innovation Plan (HB 1...,0.999721
578,237,237,237,Dodd City ISD,First,pdf,http://toolbox1.s3-website-us-west-2.amazonaws...,Microsoft Word - DCInnovationPlan (2).docx Dod...,0.999735


## Collect phrases surrounding date match. Currently, function collects 8 words before and 6 words after.

In [28]:
date_phrases = []
for text in docs_to_label.text:
    phrases = extract_dates.get_phrase_list(text = text)
    for phrase in phrases:
        date_phrases.append(phrase)
date_phrases = [str(item) for item in date_phrases]
date_phrases

['of Innovation Plan 1 I. Introduction House Bill 1842, passed during the 84th Legislative',
 'Texas Education Code. On February 21, 2017 the Ector County Independent School District',
 'and the community. On February 28, 2017, the Board appointed an eighteen',
 'March 2, 6, and 10, 2017, to discuss and draft this',
 'term of the Plan will begin with the 2017-2018 school year and terminate',
 'the Plan will begin with the 2017-2018 school year and terminate at the',
 'year and terminate at the end of the 2021-2022 school year, to',
 'terminate at the end of the 2021-2022 school year, to include anything',
 'appointment of DOI members* February 14, 2016 February Board Meeting \uf0b7 Board votes',
 'as a District of Innovation February 21, 2016 February \uf0b7 Board holds public hearing',
 'final Local Innovation Plan on April 11, 2017* February 28, 2017 March',
 'April 11, 2017* February 28, 2017 March DOI meets to draft Innovation',
 'meets to draft Innovation Plan March 2, 2017 March 

In [29]:
phrase_df = pd.DataFrame({'text':date_phrases})

In [30]:
i = 1
for item in phrase_df.text:
    print(i, '-', item)
    i = i + 1

1 - of Innovation Plan 1 I. Introduction House Bill 1842, passed during the 84th Legislative
2 - Texas Education Code. On February 21, 2017 the Ector County Independent School District
3 - and the community. On February 28, 2017, the Board appointed an eighteen
4 - March 2, 6, and 10, 2017, to discuss and draft this
5 - term of the Plan will begin with the 2017-2018 school year and terminate
6 - the Plan will begin with the 2017-2018 school year and terminate at the
7 - year and terminate at the end of the 2021-2022 school year, to
8 - terminate at the end of the 2021-2022 school year, to include anything
9 - appointment of DOI members* February 14, 2016 February Board Meeting  Board votes
10 - as a District of Innovation February 21, 2016 February  Board holds public hearing
11 - final Local Innovation Plan on April 11, 2017* February 28, 2017 March
12 - April 11, 2017* February 28, 2017 March DOI meets to draft Innovation
13 - meets to draft Innovation Plan March 2, 2017 March DOI 

In [31]:
cats = [False, False, False, False, False,
        False, False, False, False, False, 
        True, False, False, False, True,
        False, True, True, True, False,
        False, False, False, False, False,

        False, False, True, True, False,
        False, False, False, False, False,
        True, True, True, True, True,
        True, False, False, False, False,
        False, False, False, False, False,

        False, False, False, False, False,
        True, True, True, True, True,
        True, False, False, False, False,
        False,False,False,False,False,
        False, True, True, True, True,

        False, False, True, True, True,
        False, False, False, False, False,
        False, False, False, False, False,
        False, False, False, True, True,
        False,False,False,False,False,

        False, False, False, False, False,
        False, False, False, True, False,
        True,True, True]

In [32]:
phrase_df['FINALIZE'] = cats

In [33]:
phrase_df = phrase_df.sample(frac = 1)

In [35]:
print('The first n-20 of documents are used for training. Training dataset statistics:')
split = int(len(phrase_df) - 20)
print(phrase_df.head(split)['FINALIZE'].value_counts())

print('The last 20 of documents are used for testing. Testing dataset statistics:')
print(phrase_df.tail(20)['FINALIZE'].value_counts())

The first n-20 of documents are used for training. Training dataset statistics:
False    67
True     26
Name: FINALIZE, dtype: int64
The last 20 of documents are used for testing. Testing dataset statistics:
False    14
True      6
Name: FINALIZE, dtype: int64


## Train classifier

In [37]:
texts, cats = classify.reformat_cat_from_df(df = phrase_df,
                                      text_col = 'text',
                                      label_col = 'FINALIZE',
                                      category = 'FINALIZE')

In [38]:
output_dir = os.path.join(data_path, 'date_finalize_classifier')

In [39]:
classify.train_classifier_and_evaluate(texts=texts, 
                                        cats=cats,
                                        n_test = 20,
                                        model=None, 
                                        output_dir=output_dir, 
                                        n_iter=20, 
                                        categories=['FINALIZE'])

Created blank 'en' model
Using 113 examples (93 training, 20 evaluation)
Training the model...
LOSS 	  P  	  R  	  F  
5.987	0.500	0.000	0.000
4.914	0.400	0.333	0.364
3.045	0.400	0.667	0.500
3.673	0.500	0.667	0.571
2.243	0.600	1.000	0.750
1.945	0.625	0.833	0.714
1.235	0.625	0.833	0.714
1.075	0.625	0.833	0.714
0.395	0.667	1.000	0.800
0.619	0.750	1.000	0.857
0.709	0.750	1.000	0.857
0.347	0.600	1.000	0.750
0.106	0.667	1.000	0.800
0.343	0.600	1.000	0.750
0.412	0.600	1.000	0.750
0.199	0.600	1.000	0.750
0.166	0.545	1.000	0.706
0.229	0.545	1.000	0.706
0.294	0.667	1.000	0.800
0.351	0.667	1.000	0.800
No text {'FINALIZE': 0.011657624505460262}
Saved model to /Users/kylieleblancKylie/dofis/code/exemptions/../../data/date_finalize_classifier
Loading from /Users/kylieleblancKylie/dofis/code/exemptions/../../data/date_finalize_classifier
No text {'FINALIZE': 0.011657624505460262}
